# Radar Signal Intra-Pulse Modulation Recognition Based on Convolutional Denoising Auto Encoder and Deep Convolutional Neural Network

### 1 Setup

#### 1.1 Imports

In [ ]:
import torch
import torch.nn as nn
from torchinfo import summary
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
import h5py
import numpy as np
from sklearn.preprocessing import LabelEncoder
import torch.optim as optim
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tqdm import tqdm
from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
)
import cv2
import seaborn as sns
import os

#### 1.2 Device Selection

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

#### 1.3 Data Input

In [ ]:
def load_spectrogram_h5s(root_folder, mod_types):
    """
    Loads all .h5 spectrogram files from the specified modulation types
    and returns a dict structured as:
      {
          "lfm_up": {
              0: <np.ndarray>,
              1: <np.ndarray>,
              ...
          },
          ...
      }

    Parameters:
    - root_folder (str): Path to the top-level "spectrograms" directory.
    - mod_types (list): List of modulation families to include, e.g. ['FM', 'PM']

    Returns:
    - Dictionary containing all spectrograms, indexed by modulation name and integer index.
    """
    spectrogram_dict = {}

    for mod_type in mod_types:
        mod_path = os.path.join(root_folder, mod_type)
        if not os.path.exists(mod_path):
            print(f"⚠️ Warning: {mod_path} does not exist. Skipping.")
            continue

        print(f"📂 Loading from {mod_type}...")
        files = [f for f in os.listdir(mod_path) if f.endswith(".h5")]

        for file in tqdm(files, desc=f"   {mod_type}", unit="file"):
            mod_name = file[:-3]  # Strip '.h5'
            file_path = os.path.join(mod_path, file)
            spectrogram_dict[mod_name] = {}

            try:
                with h5py.File(file_path, "r") as h5f:
                    for key in h5f.keys():
                        idx = int(key)  # Convert string index to int
                        spectrogram_dict[mod_name][idx] = np.array(h5f[key])
            except Exception as e:
                print(f"❌ Failed to load {file_path}: {e}")

    return spectrogram_dict


In [ ]:
# Ensure the main directory exists
data_path = "C:/Users/scocks/Documents/hehehehehehhe/images/"
os.makedirs(data_path, exist_ok=True)

img_res = 224
img_count = 1000
snr = 0


folder_name = f"General_Images_res_{img_res}_sz_{img_count}_SNR_{snr}"
folder_path = data_path+folder_name

modulation_types = [
    "FM",
    "PM",
    "HYBRID",
]

data = load_spectrogram_h5s(folder_path, modulation_types)

##### 1.3.6 Conversion

In [ ]:
def convert_spectrogram_dict_to_xy(data_dict):
    """
    Converts a dictionary of spectrograms into (X, y) format for ML.

    Parameters:
    - data_dict: Output from load_spectrogram_h5s(), e.g.
        {
            "lfm_up": {0: np.array, 1: np.array, ...},
            "bpsk":   {0: np.array, ...},
            ...
        }

    Returns:
    - X: np.ndarray of shape (N, H, W, C)
    - y: np.ndarray of shape (N,) with string labels like 'lfm_up'
    """
    X_list = []
    y_list = []

    for label, spectros in data_dict.items():
        for idx in sorted(spectros.keys()):
            X_list.append(spectros[idx])
            y_list.append(label)

    X = np.array(X_list)
    y = np.array(y_list)

    return X, y


In [ ]:
X, y = convert_spectrogram_dict_to_xy(data)

In [ ]:
print(X.shape)
print(y.shape)

#### 1.4 Label Encoder

In [ ]:
label_encoder = LabelEncoder()

#### 1.5 Data Loader

In [ ]:
def prepare_dataloader(X, y, batch_size=32, shuffle=False, num_workers=2, device="cpu"):
    # Convert NumPy arrays to PyTorch tensors
    if isinstance(X, np.ndarray):
        X = torch.tensor(X, dtype=torch.float32)
    elif not isinstance(X, torch.Tensor):
        raise TypeError("Input X must be a NumPy array or PyTorch tensor")

    if isinstance(y, np.ndarray):
        y = torch.tensor(y, dtype=torch.long)
    elif not isinstance(y, torch.Tensor):
        raise TypeError("Labels y must be a NumPy array or PyTorch tensor")

    # Ensure X has four dimensions (N, C, H, W)
    if X.ndim == 3:  # If (N, H, W), add a channel dimension
        X = X.unsqueeze(1)  # (N, 1, H, W)
    elif X.ndim == 4 and X.shape[-1] in [1, 3]:  # (N, H, W, C) case
        X = X.permute(0, 3, 1, 2)  # Convert to (N, C, H, W)

    # Move data to the correct device
    X, y = X.to(device), y.to(device)

    # Create dataset and dataloader
    dataset = TensorDataset(X, y)
    loader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=num_workers,
        pin_memory=(device == "cuda"),
    )

    return loader

### 2 Pre-processing

In [ ]:
def pre_processing(images):
    # Resize images to 64x64 using bilinear interpolation
    resized_images = np.array(
        [cv2.resize(img, (64, 64), interpolation=cv2.INTER_LINEAR) for img in images]
    )

    # Amplitude Normalization: Dr×s (u, v) = Cr×s (u, v) / Cmax
    normalized_images = np.array(
        [img / img.max() if img.max() > 0 else img for img in resized_images]
    )

    # Convert to uint8 before grayscale conversion (as OpenCV expects 8-bit or 32-bit float images)
    normalized_images_uint8 = np.array(
        [np.uint8(img * 255) for img in normalized_images]
    )

    # Convert to grayscale (for RGB images, this will reduce the 3 channels to 1)
    grayscale_images = np.array(
        [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in normalized_images_uint8]
    )

    return grayscale_images

In [ ]:
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_pre_processed = pre_processing(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_pre_processed, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

In [ ]:
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)

print("Class Distribution in Training Set:")
for label, count in zip(unique_train, counts_train):
    print(f"Class {label}: {count} samples")

print("\nClass Distribution in Test Set:")
for label, count in zip(unique_test, counts_test):
    print(f"Class {label}: {count} samples")

#### 2.1 Pre-processing Display

In [ ]:
# selected_display = np.random.randint(len(X_train))
selected_display = 0

In [ ]:
print("Train Images shape:", X_pre_processed.shape)
print("Train Metadata shape:", y_encoded.shape)
print(f"{y[selected_display]} = {y_encoded[selected_display]}")

In [ ]:
plt.imshow(X[selected_display])
plt.show()

In [ ]:
plt.imshow(X_pre_processed[selected_display], cmap="grey")
plt.show()
print(X_pre_processed[selected_display].shape)


### 3 Training

#### 3.1 Training Setup

In [ ]:
def train_model(
    model,
    train_loader,
    device,
    criterion,
    optimizer,
    scheduler=None,  # 🔧 Optional scheduler added
    epochs=10,
    patience=3,
    min_delta=0.0,
):

    model.to(device)
    model.train()

    loss_history = []
    best_loss = float("inf")
    patience_counter = 0

    for epoch in range(epochs):
        total_loss = 0.0

        # Progress bar for each epoch
        progress_bar = tqdm(
            train_loader,
            desc=f"Epoch {epoch+1}/{epochs}",
            leave=True,
            dynamic_ncols=True,
        )

        for inputs, labels in progress_bar:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Forward pass: Ignore output_image, focus only on output_class
            _, output_class = model(inputs)

            # Classification loss
            loss = criterion(output_class, labels)

            # Backpropagation
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # Live loss display
            progress_bar.set_postfix({"Loss": f"{loss.item():.4f}"})

        # Average loss for the epoch
        avg_loss = total_loss / len(train_loader)
        loss_history.append(avg_loss)

        print(f"Epoch {epoch+1} average loss: {avg_loss:.4f}")

        # 🔄 Scheduler step
        if scheduler:
            scheduler.step()

        # Early stopping
        if avg_loss < best_loss - min_delta:
            best_loss = avg_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}")
                break

    return loss_history


##### Loss Curve

In [ ]:
def plot_loss_curve(loss_history, title="Training Loss Over Epochs"):
    epochs = len(loss_history)

    plt.figure(figsize=(8, 5))
    plt.plot(range(1, epochs + 1), loss_history, marker="o", label="Training Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(title)
    plt.legend()
    plt.grid()
    plt.show()

##### Conf Matirx

In [ ]:
def display_confusion_matrix(
    model, data_loader, device, class_names=None, title="Confusion Matrix"
):
    """
    Generate and display a normalized confusion matrix for a trained model.
    
    Parameters:
        model (torch.nn.Module): Trained PyTorch model.
        data_loader (torch.utils.data.DataLoader): DataLoader for evaluation dataset.
        device (torch.device): Device to run evaluation on (CPU/GPU).
        class_names (list, optional): List of class names. If None, uses numeric indices.
        title (str): Title of the confusion matrix plot.
    """
    # Switch model to evaluation mode
    model.to(device)
    model.eval()

    all_preds = []
    all_labels = []

    # Disable gradient calculations for inference
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            # Forward pass: Ignore output_image, focus only on output_class
            _, output_class = model(inputs)

            # Get predicted class labels
            preds = torch.argmax(output_class, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Compute confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    num_classes = cm.shape[0]
    
    # Normalize confusion matrix to percentages
    cm_normalized = cm.astype(np.float32) / cm.sum(axis=1, keepdims=True) * 100

    # If class_names isn't provided, use numeric class indices
    if class_names is None:
        class_names = [str(i) for i in range(num_classes)]

    # Plotting the confusion matrix
    plt.figure(figsize=(max(10, num_classes * 0.8), max(8, num_classes * 0.6)))  # Dynamic size
    im = plt.imshow(cm_normalized, interpolation="nearest", cmap="Blues")
    plt.title(title, fontsize=14)
    plt.colorbar(im, label="Percentage")  # Add colorbar with label

    # Create tick marks for class labels
    tick_marks = np.arange(num_classes)
    plt.xticks(tick_marks, class_names, rotation=45, ha="right", va="top", fontsize=max(8, 12 - num_classes // 5))
    plt.yticks(tick_marks, class_names, fontsize=max(8, 12 - num_classes // 5))

    # Annotate the matrix cells with percentage values
    thresh = cm_normalized.max() / 2.0
    for i in range(num_classes):
        for j in range(num_classes):
            plt.text(
                j,
                i,
                f"{cm_normalized[i, j]:.1f}",
                ha="center",
                va="center",
                color="white" if cm_normalized[i, j] > thresh else "black",
                fontsize=max(8, 12 - num_classes // 5),
            )

    plt.ylabel("True Label", fontsize=12, labelpad=10)
    plt.xlabel("Predicted Label", fontsize=12, labelpad=10)
    
    # Adjust layout with extra bottom margin for rotated labels
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2 + num_classes * 0.005)  # Dynamic bottom margin
    
    plt.show()

#### 3.2 Model

In [ ]:
class Inception1(nn.Module):
    def __init__(self):
        super(Inception1, self).__init__()

        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2), stride=2)

        self.conv1 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, stride=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=1, stride=1)
        self.conv4 = nn.Conv2d(
            in_channels=16, out_channels=16, kernel_size=3, stride=1, padding=1
        )
        self.conv5 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=2, stride=2)

        # self.depthwise_conv = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=1, stride=1, groups=64)

    def forward(self, x):

        # print("Inc1")

        x1 = self.pool1(x)

        # print("x1",x1.shape)

        x2 = self.conv1(x)
        x2 = self.conv2(x2)
        # print("x2",x2.shape)

        x3 = self.conv3(x)
        x3 = self.conv4(x3)
        x3 = self.conv5(x3)
        # print("x3",x3.shape)

        x = torch.cat((x1, x2, x3), dim=1)

        # x = self.depthwise_conv(x)

        return x

In [ ]:
class Inception2a(nn.Module):
    def __init__(self):
        super(Inception2a, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=64, out_channels=16, kernel_size=1, stride=1)

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=64, out_channels=16, kernel_size=1, stride=1)

        self.conv3 = nn.Conv2d(in_channels=64, out_channels=24, kernel_size=1, stride=1)
        self.conv4 = nn.Conv2d(
            in_channels=24, out_channels=32, kernel_size=3, stride=1, padding=1
        )  # Added padding

        self.conv5 = nn.Conv2d(in_channels=64, out_channels=24, kernel_size=1, stride=1)
        self.conv6 = nn.Conv2d(
            in_channels=24, out_channels=32, kernel_size=3, stride=1, padding=1
        )  # Added padding
        self.conv7 = nn.Conv2d(
            in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1
        )  # Added padding

    def forward(self, x):
        # print("Inc2a")

        x1 = self.conv1(x)
        # print("x1", x1.shape)

        x2 = self.pool1(x)
        x2 = self.conv2(x2)
        x2 = (
            nn.functional.pad(x2, (0, -1, 0, -1)) if x2.shape[-1] > 32 else x2
        )  # Crop if needed
        # print("x2", x2.shape)

        x3 = self.conv3(x)
        x3 = self.conv4(x3)
        # print("x3", x3.shape)

        x4 = self.conv5(x)
        x4 = self.conv6(x4)
        x4 = self.conv7(x4)
        # print("x4", x4.shape)

        x = torch.cat([x1, x2, x3, x4], 1)
        return x

In [ ]:
class Inception2b(nn.Module):
    def __init__(self):
        super(Inception2b, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=96, out_channels=24, kernel_size=1, stride=1)

        # Added padding=1 to prevent size reduction
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=24, kernel_size=1, stride=1)

        self.conv3 = nn.Conv2d(in_channels=96, out_channels=32, kernel_size=1, stride=1)
        self.conv4 = nn.Conv2d(
            in_channels=32,
            out_channels=48,
            kernel_size=(3, 1),
            stride=1,
            padding=(1, 0),
        )  # Padding added
        self.conv5 = nn.Conv2d(
            in_channels=48,
            out_channels=48,
            kernel_size=(1, 3),
            stride=1,
            padding=(0, 1),
        )  # Padding added

        self.conv6 = nn.Conv2d(in_channels=96, out_channels=32, kernel_size=1, stride=1)
        self.conv7 = nn.Conv2d(
            in_channels=32,
            out_channels=32,
            kernel_size=(3, 1),
            stride=1,
            padding=(1, 0),
        )  # Padding added
        self.conv8 = nn.Conv2d(
            in_channels=32,
            out_channels=32,
            kernel_size=(1, 3),
            stride=1,
            padding=(0, 1),
        )  # Padding added
        self.conv9 = nn.Conv2d(
            in_channels=32,
            out_channels=48,
            kernel_size=(3, 1),
            stride=1,
            padding=(1, 0),
        )  # Padding added
        self.conv10 = nn.Conv2d(
            in_channels=48,
            out_channels=48,
            kernel_size=(1, 3),
            stride=1,
            padding=(0, 1),
        )  # Padding added

    def forward(self, x):

        x1 = self.conv1(x)
        # print("x1", x1.shape)  # Should be [1, 24, 16, 16]

        x2 = self.pool1(x)
        x2 = self.conv2(x2)
        x2 = (
            nn.functional.pad(x2, (0, -1, 0, -1)) if x2.shape[-1] > 16 else x2
        )  # Crop if needed
        # print("x2", x2.shape)  # Should be [1, 24, 16, 16]

        x3 = self.conv3(x)
        x3 = self.conv4(x3)
        x3 = self.conv5(x3)
        # print("x3", x3.shape)  # Should be [1, 48, 16, 16]

        x4 = self.conv6(x)
        x4 = self.conv7(x4)
        x4 = self.conv8(x4)
        x4 = self.conv9(x4)
        x4 = self.conv10(x4)
        # print("x4", x4.shape)  # Should be [1, 48, 16, 16]

        x = torch.cat([x1, x2, x3, x4], 1)
        return x

In [ ]:
class Inception3a(nn.Module):
    def __init__(self):
        super(Inception3a, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=144, out_channels=32, kernel_size=1, stride=1
        )

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1, padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=144, out_channels=32, kernel_size=1, stride=1
        )

        self.conv3 = nn.Conv2d(
            in_channels=144, out_channels=48, kernel_size=1, stride=1
        )
        self.conv4 = nn.Conv2d(
            in_channels=48,
            out_channels=48,
            kernel_size=(3, 1),
            stride=1,
            padding=(1, 0),
        )  # Added padding
        self.conv5 = nn.Conv2d(
            in_channels=48,
            out_channels=48,
            kernel_size=(1, 3),
            stride=1,
            padding=(0, 1),
        )  # Added padding

        self.conv6 = nn.Conv2d(
            in_channels=144, out_channels=32, kernel_size=1, stride=1
        )
        self.conv7 = nn.Conv2d(
            in_channels=32, out_channels=48, kernel_size=3, stride=1, padding=1
        )  # Added padding
        self.conv8 = nn.Conv2d(
            in_channels=48,
            out_channels=16,
            kernel_size=(1, 3),
            stride=1,
            padding=(0, 1),
        )  # Added padding
        self.conv9 = nn.Conv2d(
            in_channels=48,
            out_channels=16,
            kernel_size=(3, 1),
            stride=1,
            padding=(1, 0),
        )  # Added padding

    def forward(self, x):
        x1 = self.conv1(x)
        # print("x1:", x1.shape)  # [1, 32, 8, 8]

        x2 = self.pool1(x)
        x2 = self.conv2(x2)
        x2 = (
            nn.functional.pad(x2, (0, -1, 0, -1)) if x2.shape[-1] > 8 else x2
        )  # Crop if needed
        # print("x2:", x2.shape)  # [1, 32, 8, 8]

        x3_pre_split = self.conv3(x)
        x3 = self.conv4(x3_pre_split)
        x4 = self.conv5(x3_pre_split)
        # print("x3:", x3.shape)  # [1, 48, 8, 8]
        # print("x4:", x4.shape)  # [1, 48, 8, 8]

        x5_pre_split = self.conv6(x)
        x5_pre_split = self.conv7(x5_pre_split)
        x5 = self.conv8(x5_pre_split)
        x6 = self.conv9(x5_pre_split)
        # print("x5:", x5.shape)  # [1, 16, 8, 8]
        # print("x6:", x6.shape)  # [1, 16, 8, 8]

        x = torch.cat([x1, x2, x3, x4, x5, x6], 1)
        # print("Final Output:", x.shape)  # Should be [1, 192, 8, 8]
        return x

In [ ]:
class Inception3b(nn.Module):
    def __init__(self):
        super(Inception3b, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=192, out_channels=32, kernel_size=1, stride=1
        )

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1, padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=192, out_channels=32, kernel_size=1, stride=1
        )

        self.conv3 = nn.Conv2d(
            in_channels=192, out_channels=48, kernel_size=1, stride=1
        )
        self.conv4 = nn.Conv2d(
            in_channels=48,
            out_channels=64,
            kernel_size=(3, 1),
            stride=1,
            padding=(1, 0),
        )  # Added padding
        self.conv5 = nn.Conv2d(
            in_channels=48,
            out_channels=64,
            kernel_size=(1, 3),
            stride=1,
            padding=(0, 1),
        )  # Added padding

        self.conv6 = nn.Conv2d(
            in_channels=192, out_channels=32, kernel_size=1, stride=1
        )
        self.conv7 = nn.Conv2d(
            in_channels=32, out_channels=48, kernel_size=3, stride=1, padding=1
        )  # Added padding
        self.conv8 = nn.Conv2d(
            in_channels=48,
            out_channels=32,
            kernel_size=(1, 3),
            stride=1,
            padding=(0, 1),
        )  # Added padding
        self.conv9 = nn.Conv2d(
            in_channels=48,
            out_channels=32,
            kernel_size=(3, 1),
            stride=1,
            padding=(1, 0),
        )  # Added padding

    def forward(self, x):
        x1 = self.conv1(x)
        # print("x1:", x1.shape)  # [1, 32, 8, 8]

        x2 = self.pool1(x)
        x2 = self.conv2(x2)
        x2 = (
            nn.functional.pad(x2, (0, -1, 0, -1)) if x2.shape[-1] > 4 else x2
        )  # Crop if needed
        # print("x2:", x2.shape)  # [1, 32, 8, 8]

        x3_pre_split = self.conv3(x)
        x3 = self.conv4(x3_pre_split)
        x4 = self.conv5(x3_pre_split)
        # print("x3:", x3.shape)  # [1, 48, 8, 8]
        # print("x4:", x4.shape)  # [1, 48, 8, 8]

        x5_pre_split = self.conv6(x)
        x5_pre_split = self.conv7(x5_pre_split)
        x5 = self.conv8(x5_pre_split)
        x6 = self.conv9(x5_pre_split)
        # print("x5:", x5.shape)  # [1, 16, 8, 8]
        # print("x6:", x6.shape)  # [1, 16, 8, 8]

        x = torch.cat([x1, x2, x3, x4, x5, x6], 1)
        # print("Final Output:", x.shape)  # Should be [1, 192, 8, 8]
        return x

##### Deconv

In [ ]:
class deconv_block(nn.Module):
    def __init__(self):
        super(deconv_block, self).__init__()

        # Deconv 1: Input (4x4x192) -> Output (8x8x128)
        self.deconv1 = nn.ConvTranspose2d(
            in_channels=192,
            out_channels=128,
            kernel_size=3,
            stride=2,
            padding=1,
            output_padding=1,
        )

        # Deconv 2: Input (8x8x128) -> Output (16x16x96)
        self.deconv2 = nn.ConvTranspose2d(
            in_channels=128,
            out_channels=96,
            kernel_size=3,
            stride=2,
            padding=1,
            output_padding=1,
        )

        # Deconv 3: Input (16x16x96) -> Output (32x32x48)
        self.deconv3 = nn.ConvTranspose2d(
            in_channels=96,
            out_channels=48,
            kernel_size=3,
            stride=2,
            padding=1,
            output_padding=1,
        )

        # Deconv 4: Input (32x32x48) -> Output (64x64x1)
        self.deconv4 = nn.ConvTranspose2d(
            in_channels=48,
            out_channels=1,
            kernel_size=3,
            stride=2,
            padding=1,
            output_padding=1,
        )

    def forward(self, x):

        x = self.deconv1(x)
        x = self.deconv2(x)
        x = self.deconv3(x)
        x = self.deconv4(x)

        return x

##### Main Model

In [ ]:
class MainModel(nn.Module):
    def __init__(self, num_classes):
        super(MainModel, self).__init__()

        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1
        )

        self.inc1 = Inception1()

        self.inc2a = Inception2a()

        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.inc2b = Inception2b()

        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.inc3a = Inception3a()

        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.deconv = deconv_block()

        self.inc3b = Inception3b()

        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.fc = nn.Linear(in_features=1024, out_features=num_classes)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):

        x = self.conv1(x)

        x = self.inc1(x)

        x = self.inc2a(x)

        x = self.pool1(x)

        x = self.inc2b(x)

        x = self.pool2(x)

        x = self.inc3a(x)

        x = self.pool3(x)

        output_image = self.deconv(x)

        x = self.inc3b(x)

        x = self.pool4(x)

        x = torch.flatten(x, 1)
        x = self.fc(x)
        output_class = x
        # output_class = self.softmax(output_class)

        return output_image, output_class

##### Model Summary

In [ ]:
model = MainModel(6)
# summary(model , input_size=(1, 1, 64, 64))

#### 3.3 Actual Training

In [ ]:
# Prepare DataLoaders
train_loader = prepare_dataloader(
    X_train,
    y_train,
    batch_size=32,
    shuffle=True,
)

In [ ]:
num_classes = len(np.unique(y_encoded))

model = MainModel(num_classes=num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss().to(device)

In [ ]:
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=45, gamma=0.1)

In [ ]:
# Train the model
epoch_count = 1
loss_history = train_model(
    model=model,
    train_loader=train_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    device=device,
    epochs=epoch_count,
    patience=50,
)

In [ ]:
plot_loss_curve(loss_history)

In [ ]:
display_confusion_matrix(model, train_loader, device)

#### 3.4 Save Model to File

In [ ]:
mds = modulation_types[0]
if len(modulation_types) == 3:
    mds = "ALL"

model_file_name = f"CDAE_DCNN_model_e{epoch_count}_lr{learning_rate}_snr_{snr}_mds_{mds}"

In [ ]:
torch.save(model, model_file_name + ".pth")

### 4 Testing

In [ ]:
def evaluate_model(model, test_loader, label_encoder, device):
    """
    Evaluates the trained model and displays accuracy, confusion matrix, F1-score,
    and one output image per class.

    Args:
        model: Trained PyTorch model.
        test_loader: DataLoader for test set.
        label_encoder: Label encoder to decode class names.
        device: 'cuda' or 'cpu' where evaluation happens.
    """
    model.to(device)  # Ensure model is on correct device
    model.eval()  # Set to evaluation mode

    y_true = []
    y_pred = []
    correct = 0
    total = 0

    # Dictionary to store one image per class
    class_images = {}

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            # Model returns (output_image, output_class)
            output_image, output_class = model(inputs)

            # Get predicted class (argmax over logits)
            preds = torch.argmax(output_class, dim=1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

            y_true.extend(labels.cpu().tolist())  # Move to CPU for metrics
            y_pred.extend(preds.cpu().tolist())

            # Store one output image per class
            for img, pred_class in zip(output_image, preds):
                pred_class = pred_class.item()
                if pred_class not in class_images:
                    class_images[pred_class] = img.cpu()

    # Compute Accuracy
    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")

    # Compute & Display Confusion Matrix
    class_names = label_encoder.classes_  # Decode label names
    cm = confusion_matrix(y_true, y_pred)
    # Normalize confusion matrix to percentages
    cm_normalized = cm.astype(np.float32) / cm.sum(axis=1, keepdims=True) * 100
    num_classes = len(class_names)

    # Plot confusion matrix
    fig, ax = plt.subplots(figsize=(max(10, num_classes * 0.8), max(8, num_classes * 0.6)))  # Dynamic size
    disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=class_names)
    disp.plot(cmap="Blues", values_format=".1f", ax=ax)  # Use 1 decimal place for percentages

    # Adjust x-axis label alignment and font sizes
    ax.set_xticklabels(class_names, rotation=45, ha="right", va="top", fontsize=max(8, 12 - num_classes // 5))
    ax.set_yticklabels(class_names, rotation=0, fontsize=max(8, 12 - num_classes // 5))
    ax.set_xlabel("Predicted Label", fontsize=12, labelpad=10)
    ax.set_ylabel("True Label", fontsize=12, labelpad=10)
    ax.set_title("Confusion Matrix (Percentage)", fontsize=14)

    # Adjust layout with extra bottom margin for rotated labels
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2 + num_classes * 0.005)  # Dynamic bottom margin

    plt.show()

    # Print Classification Report (Precision, Recall, F1-score)
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Display one output image per class
    print("\nSample Output Images for Each Class:")
    num_classes = len(class_names)
    plt.figure(figsize=(15, 5))
    for idx, (class_idx, img) in enumerate(class_images.items()):
        plt.subplot(1, num_classes, idx + 1)
        plt.imshow(img.permute(1, 2, 0))  # Assuming (C, H, W) format
        plt.title(class_names[class_idx])
        plt.axis("off")
    plt.tight_layout()
    plt.show()

In [ ]:
# Prepare DataLoaders
test_loader = prepare_dataloader(
    X_test,
    y_test,
    batch_size=32,
)

In [ ]:
# Evaluate the model
evaluate_model(model, test_loader, label_encoder, device)